# Vector database

## Global setup

In [1]:
# Global constants
OLLAMA_EMBEDDING_URL="http://localhost:11434/api/embed"
EMBEDDING_MODEL="nomic-embed-text"

In [2]:
# Global imports
import json
import requests

In [3]:
# Global utility functions
def get_embedding(session: requests.Session, model_name: str, text_input):
    response = session.post(
        OLLAMA_EMBEDDING_URL,
        data=json.dumps({
            "model": model_name,
            "input": text_input
        })
    )
    return response.json()

In [4]:
session = requests.Session()

## Try embeddings

In [7]:
response = get_embedding(session, EMBEDDING_MODEL, ["Hello World!"])
response["embeddings"]

[[0.0053903093,
  -0.0026179042,
  -0.17719288,
  -0.011143332,
  -0.013133138,
  0.0702707,
  -0.009469159,
  -0.026641931,
  -0.009747269,
  -0.058955394,
  0.013530472,
  0.055808485,
  0.022312537,
  0.06133417,
  0.024879402,
  -0.04767762,
  0.012098431,
  -0.052931823,
  -0.043903783,
  0.020416837,
  -0.02836235,
  -0.081815675,
  0.0026377046,
  0.032095633,
  0.10631707,
  0.010502971,
  -0.050546005,
  0.061973784,
  0.0050363615,
  0.0041382113,
  -0.01360648,
  -0.0063092876,
  -0.0014158383,
  0.029270498,
  0.060902916,
  -0.00017745754,
  0.029295564,
  0.01138837,
  0.021856742,
  -0.012949824,
  0.0058418657,
  -0.012409977,
  0.013261852,
  -0.000922938,
  0.07715618,
  -0.016976612,
  -0.0023586017,
  -0.0167604,
  0.07356576,
  -0.037564773,
  -0.041481137,
  0.0018473217,
  0.000385279,
  0.05638736,
  0.060303554,
  0.01692413,
  0.045276172,
  -0.049780067,
  0.016918506,
  0.031175999,
  0.042276923,
  0.05287051,
  0.030832794,
  0.053609595,
  -0.005021142,
 

In [8]:
len(response["embeddings"][0])

768

## Load vector database

In [9]:
from qdrant_client import QdrantClient

# Create the vector database client
client = QdrantClient("http://localhost:6333")

Read all the files and get the embeddings so we can store the information in the vector database.

In [10]:
from pathlib import Path

data = []

path = Path("./contents")
for file in path.iterdir():
    if file.is_file():
        data.append(file.read_text())

In [11]:
data

['Menu of the restaurant\n\n1. Patatas Bravas \nDescription: Crispy fried potatoes topped with a spicy tomato sauce and a creamy garlic aioli. \nIngredients: Potatoes, tomato sauce, garlic, olive oil, mayonnaise, paprika.\n\n2. Paella de Mariscos \nDescription: A traditional Spanish rice dish filled with a variety of fresh seafood. \nIngredients: Rice, shrimp, mussels, clams, squid, saffron, garlic, olive oil, bell peppers, tomatoes.\n\n3. Gazpacho Andaluz \nDescription: A refreshing cold tomato soup perfect for a hot day. \nIngredients: Tomatoes, cucumbers, bell peppers, onions, garlic, olive oil, vinegar, salt.\n\n4. Churros con Chocolate \nDescription: Fried dough pastries dusted with sugar and served with a rich chocolate dipping sauce. \nIngredients: Flour, water, sugar, cinnamon, dark chocolate, milk.\n\n5. Tortilla Española \nDescription: A classic Spanish omelette made with potatoes and onions. \nIngredients: Eggs, potatoes, onions, olive oil, salt.',
 "About Me\n\nHi there! I'

In [12]:
data_embeddings = get_embedding(session, EMBEDDING_MODEL, data)
data_embeddings["embeddings"]

[[-0.01966265,
  0.09049757,
  -0.19104382,
  0.027474608,
  0.020606115,
  -0.02185858,
  -0.011541874,
  -0.024420936,
  -0.009147898,
  -0.01961362,
  0.0072602765,
  0.02089015,
  0.0612601,
  0.048945557,
  -0.009947001,
  -0.0679581,
  0.006497176,
  -0.016915346,
  -0.00077573647,
  0.100224,
  0.045754943,
  0.0015781198,
  -0.04697007,
  -0.036645606,
  0.00033288254,
  0.07069832,
  -0.055915304,
  0.1189572,
  -0.020754334,
  -0.057842575,
  -0.042057812,
  -0.055680502,
  0.0041514663,
  0.039379794,
  0.032272734,
  -0.061857734,
  0.061021797,
  0.055727106,
  -0.013566586,
  -0.014171825,
  0.027281918,
  0.061673835,
  0.021932252,
  0.012101878,
  0.00906209,
  -0.013307813,
  0.028033543,
  0.026093878,
  -0.039128404,
  -0.027201744,
  0.022599049,
  -0.013791466,
  -0.007266992,
  -0.05690129,
  -0.05008825,
  -0.020544674,
  0.016507093,
  0.03548959,
  0.07628417,
  0.017085457,
  0.07667007,
  0.017442958,
  -0.012877312,
  0.07823843,
  0.017231584,
  -0.0272051

Create the collection to store the embeddings

In [13]:
from qdrant_client.models import Distance, VectorParams

client.create_collection(
    collection_name="restaurant",
    vectors_config=VectorParams(size=768, distance=Distance.COSINE)
)

True

Store the embeddings with a payload, which is the text you want to store.

In [14]:
from qdrant_client.models import PointStruct

points = []
for idx, (text, embedding) in enumerate(zip(data, data_embeddings["embeddings"])):
    points.append(PointStruct(
        id=idx,
        vector=embedding,
        payload={"text": text}
    ))

client.upsert(
    collection_name="restaurant",
    points=points
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

Query the vector database, for the most similar result.

In [17]:
query_embedding = get_embedding(session, EMBEDDING_MODEL, ["Who's your creator?"])
query_embedding

{'model': 'nomic-embed-text',
 'embeddings': [[-0.013410551,
   0.0011850083,
   -0.119074404,
   0.00041199705,
   0.0065833475,
   -0.002318786,
   -0.026585482,
   -0.009772712,
   -0.039829504,
   -0.05508692,
   -0.055414893,
   0.0039134035,
   0.111942254,
   0.018973041,
   0.0058583366,
   -0.021987583,
   0.018205317,
   -0.05552228,
   0.011898238,
   0.041692585,
   -0.06676032,
   -0.106055826,
   0.00031556355,
   0.014866624,
   0.15605043,
   -0.057539713,
   -0.042624127,
   0.014962885,
   -0.015858239,
   0.010693524,
   0.024934758,
   0.069302954,
   -0.027640145,
   0.008316822,
   0.00045487666,
   0.01166585,
   -0.01268492,
   0.03222113,
   0.029556496,
   0.030272506,
   -0.027588133,
   0.016522693,
   -0.07837891,
   -0.09621746,
   0.0042939424,
   -0.03341567,
   0.04024863,
   -0.0020498217,
   0.030979393,
   -0.027903903,
   -0.011470823,
   -0.041599154,
   -0.01068152,
   0.021092724,
   0.06147705,
   0.009369824,
   -0.040970888,
   0.01422807,
   

In [18]:
results = client.search(
    collection_name="restaurant",
    query_vector=query_embedding["embeddings"][0],
    limit=1
)
results

[ScoredPoint(id=1, version=0, score=0.5372231, payload={'text': "About Me\n\nHi there! I'm Jamón, your friendly chatbot here to help with all things related to our restaurant. If you have any questions or need assistance, I'm here for you. My talented creator, Eric, brought me to life to ensure you have the best experience possible. Let's make your visit delightful! "}, vector=None, shard_key=None, order_value=None)]